In [48]:
import json
import re

class fubot_config():
    class feat_group():
        class feat_type():
            lut_C = {'w': 1, 'x':2, 'y':3, 'z':4}
            lut_C_inv = list(lut_C.keys())
            lut_S = {'l': 1, 'w':2}
            lut_T = {'pos': 1, 'qrot':2, 'rot':3}

            def __init__(self, S, T, C):              
                self.S = self.lut_S[S]
                self.T = self.lut_T[T]
                self.C = self.lut_C[C]
                self.C_id = self.C - (1 if self.T == 2 else 2)
                self.valid = (self.S * self.T * self.C) > 0

            def getValue(self, v_local, v_world):
                return v_local[self.C_id] if self.S == 1 else v_world[self.C_id]

            def getPostFix(self):
                pt = 'p' if self.T == 1 else 'r'
                pc = self.lut_C_inv[self.C - 1]
                
                return f'_{pt}{pc}'

        def __init__(self, feature_config):
            self.name = None
            self.bone_name = None
            self.elements = []
            self.size = 0
            self.rgx_el = re.compile(r'(.+)\[(.+)\]_(.)')
            self.__parse__(feature_config)

        def __parse__(self, feature_config):
            self.name = feature_config[0]
            self.bone_name = feature_config[1] if len(feature_config[1]) > 0 else self.name

            #Feature Elements
            for f_el in feature_config[2]:
                m = self.rgx_el.match(f_el)
                g = m.groups()
                if len(g) is not 3:
                    print(f'Invalid feature element found. ({self.name} >> {f_el})')
                    continue

                for comp in g[1]:
                    ft = self.feat_type(g[2], g[0], comp)
                    if not ft.valid:
                        print(f'Invalid feature element found. ({self.name} >> {f_el} [{g[2]}, {g[0]}, {comp}])')
                        continue

                    self.elements.append(ft)
            
            self.size = len(self.elements)

    def __init__(self, config_path):
        self.features_output:self.feat_group = []
        self.features_output_header = []
        self.features_output_size = 0

        self.features_input:self.feat_group = []
        self.features_input_header = []
        self. features_input_size = 0

        self.__loadConfig__(config_path)

    def __loadConfig__(self, config_path):
        with open(config_path) as f:
            config_json = json.loads(f.read())

        #input features
        features = config_json['training']['input_features']
        if features is not None:
            for feat in features:
                fg = self.feat_group(feat)
                self.features_input.append(fg)
                self.features_input_size += fg.size

                #Create Header
                for ft in fg.elements:
                    self.features_input_header.append(f'{fg.bone_name}{ft.getPostFix()}')

                

        #output features
        features = config_json['training']['output_features']
        if features is not None:
            for feat in features:
                fg = self.feat_group(feat)
                self.features_output.append(fg)
                self.features_output_size += fg.size

                #Create Header
                for ft in fg.elements:
                    self.features_output_header.append(f'{fg.bone_name}{ft.getPostFix()}')

config = fubot_config('../samples/generator_config.json')
print(config.features_input_header)

['Head_px', 'Head_py', 'Head_pz', 'Head_rw', 'Head_rx', 'Head_ry', 'Head_rz', 'Hips_px', 'Hips_py', 'Hips_pz', 'Hips_rw', 'Hips_rx', 'Hips_ry', 'Hips_rz', 'LeftHand_px', 'LeftHand_py', 'LeftHand_pz', 'LeftHand_rw', 'LeftHand_rx', 'LeftHand_ry', 'LeftHand_rz', 'RightHand_px', 'RightHand_py', 'RightHand_pz', 'RightHand_rw', 'RightHand_rx', 'RightHand_ry', 'RightHand_rz', 'LeftToes_px', 'LeftToes_py', 'LeftToes_pz', 'LeftToes_rw', 'LeftToes_rx', 'LeftToes_ry', 'LeftToes_rz', 'RightToes_px', 'RightToes_py', 'RightToes_pz', 'RightToes_rw', 'RightToes_rx', 'RightToes_ry', 'RightToes_rz']


In [35]:
from collections import OrderedDict
from bvhtoolbox import Bvh
import transforms3d as t3d
from math import radians
import numpy as np
import pandas as pd

class bvh_skeleton_node:
    def __init__(self, name, offset):
        self.children = []
        self.parent:bvh_skeleton_node = None

        self.name = name
        self.local_position = offset
        self.local_rotation_mat = None
        self.local_rotation_quat = [0,0,0,0]
        self.local_transform = None

        self.world_position = None
        self.world_rotation_mat = None
        self.world_rotation_quat = None
        self.world_transform = None

    def CalculateWorld(self):
        self.local_transform = t3d.affines.compose(self.local_position, self.local_rotation_mat, [1,1,1])

        if self.parent != None:
            self.world_transform = self.parent.world_transform.dot(self.local_transform)
        else:
            self.world_transform = self.local_transform

        self.world_position, self.world_rotation_mat, _, _ = t3d.affines.decompose(self.world_transform)
        self.world_rotation_quat = t3d.quaternions.mat2quat(self.world_rotation_mat)

        for child in self.children:
            child.CalculateWorld()

    def SetLocalRotation(self, new_rotation):
        self.local_rotation_quat = new_rotation
        self.UpdateLocalRotation()

    def UpdateLocalRotation(self):
        self.local_rotation_mat = t3d.quaternions.quat2mat(self.local_rotation_quat)

    def AddChild(self, child):
        self.children.append(child)
        child.parent = self
        return child #Enables chaining

    def print_tree(self):
        if self.parent is None:
            print(f'{self.name} : ROOT ')
        else:
            print(f'{self.name} : {self.parent.name}')

        print(f'\t{self.name}_world {{{self.world_position}, {self.world_rotation_quat}}}')
        print(f'\t{self.name}_local {{{self.local_position}, {self.local_rotation_quat}}}')

        for child in self.children:
            child.print_tree()

In [49]:
class bvh_skeleton: 
    def __init__(self, bvh_filepath):
        self.root = None
        self.nodes = OrderedDict()
        self.nodes_flat = []
        self.fp_bvh = bvh_filepath
        self.bvh:Bvh = None

        self.__loadSkeleton()

    def __loadSkeleton(self):
        with open(self.fp_bvh) as f:
            self.bvh = Bvh(f.read())
        
        joint_names = self.bvh.get_joints_names() 
        skDef = OrderedDict()
        for joint_name in joint_names:
            j = self.bvh.get_joint(joint_name)
            j_parent = j.parent.value[1] if len(j.parent.value) > 1 else None
            j_offset = [float(val) for val in j['OFFSET']]
            skDef[joint_name] = (j_offset, j_parent)

        #Construct Skeleton
        self.nodes.clear()
        for key, value in skDef.items():
            value[0][0] = -value[0][0]
            new_node = bvh_skeleton_node(key, value[0])
            self.nodes[key] = new_node

            if value[1] != None:
                self.nodes[value[1]].AddChild(new_node)
            else:
                self.root = new_node

        self.nodes_flat = list(self.nodes.values())

    def __convertToUnityPos(self, v, y_up = True):
        if y_up:
            return v * [-1,1,1]

        return v * [-1,1,-1]

    def __convertToUnityQuat(self, e, y_up = True):
        qZ = t3d.quaternions.axangle2quat([0,0,1], radians(e[0]), True) #FORWARD/Z
        qX = t3d.quaternions.axangle2quat([1,0,0], radians(e[1]), True) #RIGHT/X
        qY = t3d.quaternions.axangle2quat([0,1,0], radians(e[2]), True) #UP/Y
        q = t3d.quaternions.qmult(t3d.quaternions.qmult(qZ , qX) , qY)
        #q /= t3dq.qnorm(q)

        if y_up:
            return q * [1,1,-1,-1]
        
        return q * [1,1,1,-1]

    def __getFeature(self, joint_name, ft:fubot_config.feat_group.feat_type):
        joint = self.nodes[joint_name]
        val = None

        if ft.T == 1: #POS
            return ft.getValue(joint.local_position, joint.world_position)
        if ft.T == 2: #QROT
            return ft.getValue(joint.local_rotation_quat, joint.world_rotation_quat)
        if ft.T == 3: #ROT (euler)
            print('Euler rotations not yet supported')
            return 0


    def set_motion_frame(self,index):
        frame = self.bvh.frames[index]
        
        #semi-hardcoded
        self.root.local_position = self.__convertToUnityPos(np.array(frame[0:3]).astype(np.float))

        for idx, n in enumerate(self.nodes_flat):
            i = 3 + (idx*3)
            q = self.__convertToUnityQuat(np.array(frame[i:i+3]).astype(np.float))
            n.SetLocalRotation(q)

        self.root.CalculateWorld()

    def extract_features(self, config:fubot_config):
        #Input Features
        features_input = np.empty(config.features_input_size)
        feature_id = 0
        for fg in config.features_input:
            for ft in fg.elements:
                features_input[feature_id] = self.__getFeature(fg.bone_name, ft)
                feature_id += 1

        #Output Features
        features_output = np.empty(config.features_output_size)
        feature_id = 0
        for fg in config.features_output:
            for ft in fg.elements:
                features_output[feature_id] = self.__getFeature(fg.bone_name, ft)
                feature_id += 1

        return features_input, features_output

    def convert_to_training_set(self, config:fubot_config):
        
        train_input = None
        train_output = None

        for i in range(len(self.bvh.frames)):
            self.set_motion_frame(i)
            f_in, f_out = self.extract_features(config)

            if train_input is None:
                train_input = f_in
                train_output = f_out
            else:
                train_input = np.vstack([train_input, f_in])
                train_output = np.vstack([train_output, f_out])
        
        print(train_input.shape)
        print(len(config.features_input_header))
        print(train_output.shape)
        print(len(config.features_output_header))


        pd.DataFrame(data=train_input, columns=config.features_input_header).to_csv('temp_input.csv', index=False)   
        pd.DataFrame(data=train_output, columns=config.features_output_header).to_csv('temp_output.csv', index=False)

        
        




skeleton = bvh_skeleton('../samples/beatsaber_0_1_b.bvh')
config = fubot_config('../samples/generator_config.json')

skeleton.set_motion_frame(0)
skeleton.convert_to_training_set(config)

(300, 42)
42
(300, 91)
91
